In [107]:
import pandas as pd
from openai import OpenAI, Embedding
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import os


In [120]:
df = pd.read_csv("../11_20_26_27_articles_combined.csv") 
queries = pd.read_csv("../queries_for_documents.csv")
#openai_client = OpenAI(api_key = 'insert-api-key')

In [13]:
print(df.head())

           title                                                url  \
0  Yahoo Finance  https://finance.yahoo.com/news/threads-now-let...   
1  Yahoo Finance  https://finance.yahoo.com/news/elon-musk-calls...   
2  Yahoo Finance  https://finance.yahoo.com/news/tpg-angelo-gord...   
3  Yahoo Finance  https://finance.yahoo.com/news/microsoft-faces...   
4  Yahoo Finance  https://finance.yahoo.com/news/crowdstrike-fal...   

                          date  \
0  Wed, Nov 27, 2024, 12:15 PM   
1   Wed, Nov 27, 2024, 1:34 PM   
2  Wed, Nov 27, 2024, 11:23 AM   
3  Wed, Nov 27, 2024, 12:58 PM   
4   Wed, Nov 27, 2024, 6:40 AM   

                                             content  
0  Threads is rolling out a redesign that keeps y...  
1  Billionaire Elon Musk, a key adviser to the in...  
2  (Bloomberg) -- Americans borrowing against the...  
3  (Bloomberg) -- The US Federal Trade Commission...  
4  (Bloomberg) -- CrowdStrike Holdings Inc. fell ...  


In [82]:
print(queries.head())

                                               query  doc_index
0  Why is Australia's monetary policy still seen ...        551
1  What's Palo Alto Networks' revenue forecast fo...        400
2  How could institutional investments affect Bit...        377
3  Hey, I've been hearing that some AI companies ...        240
4  What impact could increased tariffs have on th...        516


In [14]:
from openai import OpenAI, Embedding


def summarize_doc(openai_client, text):
    query = "Summarize this into less than 10 words"

    prompt = f"Context:\n{text}\n\nQuestion: {query}\nAnswer:"


    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
        {"role": "user", "content": prompt}
        ]
    )

    print(completion.choices[0].message)

In [15]:
def preprocess_text(text):
    """Preprocess a single text input (either document or query)."""
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')  # Add custom stop words if needed
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # Replace punctuation with spaces
    text_nopunct = text.translate(translator)  # Remove punctuation
    words_list = text_nopunct.strip().split()  # Tokenize into words
    filtered_words = [word.lower() for word in words_list if word.lower() not in stop_words and len(word) > 1]
    return filtered_words

def preprocess_df(df):
    """Preprocess a DataFrame column containing document content."""
    df["content"] = df["content"].apply(preprocess_text)
    return df

def preprocess_query(query):
    """Preprocess a query string."""
    return preprocess_text(query)

In [125]:
import bm25
import importlib

# Reload the module after making changes
importlib.reload(bm25)

# Initialize the BM25 instance

def rank_query_bm25(bm25, query, ground_truth_index, top_k=5, summarize_docs=True, only_scores=False):
    # Display initial document details

    if not only_scores:
        print("=" * 50)
        print("📄 Document Sample:")
        print(df.head())
        print("=" * 50)
        print(f"📏 Total Documents: {len(bm25_instance.doc_lengths)}")
    print(f"Preprocessed query: {query} with doc_index {ground_truth_index}")
    # Rank documents for a given query
    ranked_indices, scores = bm25_instance.rank(query)

    ranked_indices = ranked_indices[:top_k]
    scores = scores[:top_k]

    if not only_scores:
        print("\n🏆 Ranking Results:")
        print("=" * 50)
        for i, (doc_index, score) in enumerate(zip(ranked_indices, scores), start=1):
            print(f"⭐ Rank {i}:")
            print(f"   - Document Index: {doc_index}")
            print(f"   - BM25 Score: {score:.4f}")
            doc_content = df.iloc[doc_index]["content"]
            if (summarize_docs):
                summmary = summarize_doc(openai_client, doc_content)
                print(f"   - Document Content:\n{summmary}")
            else:
                print(f"   - Document Content:\n{doc_content}")
            print("-" * 50)
    return ranked_indices, scores

    
    


In [79]:
query = "Why is Australia's monetary policy still seen as powerful even though a lot of households have high mortgage debt?" #551
#28 15
query2 = "What's Palo Alto Networks' revenue forecast for the second quarter, and how does it stack up against what analysts expected?" #400
#39 17
query3= "How could institutional investments affect Bitcoin's price through 2025?" #377
#18 13
query4 = "Hey, I've been hearing that some AI companies are getting into hardware. What challenges do these companies face when trying to bring AI devices to market?" #240
#21.4 20.9
query5 = "What impact could increased tariffs have on the cost of goods for companies, and how might that affect consumers?" #516
#17.2, only third, highest 17.8
query6 = "Why is the U.S. government reducing Intel's chips grant, and what factors are contributing to the change?" #345
#22.3 20.9
query7 = "What is Israel considering regarding a cease-fire with Hezbollah, and what would the terms be?" #213
#28 22
query8 = "How have Baidu's shares performed this year, and what’s the outlook for its upcoming earnings report?" #370
#14.3 13.0
query9 = "What challenges does the U.S. face in reaching an agreement over the future of Colorado River water use, and what are the possible solutions being discussed?" #554
#34 12
query10 = "Why are creditors frustrated with the stalled auction of Citgo Petroleum shares?" #91
#34 29

query11 = "How did Zoom's third-quarter results compare to analyst expectations, and what challenges is the company facing in maintaining its customer base?"  #51

query12 = "How do geopolitical risks, valuation mismatches, and economic conditions affect the IPO landscape in Spain?" #299

query13 = "Why is Ford planning to reduce its workforce in Europe and the U.K., and what are the challenges it faces in the electric vehicle market?" #567

query14 = "What’s going on with Ricardo Salinas Pliego’s financial situation, and why is Grupo Elektra’s stock trading halt causing so many issues?" #234

query15 = "Why do experts believe Russia may not be able to sustain its war in Ukraine beyond 2025?" #354

query16 = "What are the key players in the brain-computer interface market, and how are they competing to be the first to launch commercially?" #519

query17 = "What strategies are airlines using to attract premium passengers, and how is this affecting their financial performance?" #495

query18 = "How is Pocket FM using AI to expand its content library and improve production efficiency?" #28

query19 = "What is SoftBank’s plan to acquire shares in OpenAI, and how does this fit into its broader investment strategy in AI?" #55

query20 = "What factors are influencing the FHFA’s decision to raise the conforming loan limit, and what does this mean for the real estate market?" #170

query21 = "How does New Lantern plan to disrupt the radiology software market, and what impact could it have on the industry?" #421

query22 = "How is Dell’s AI server business performing, and what impact have delays in Nvidia’s Blackwell AI chips had on its revenue?" #58

query23 = "How did the potential cease-fire between Israel and Hezbollah affect oil prices?" #167

query24 = "What impact is the decline in recession fears and growing optimism about jobs and finances having on consumer confidence and spending patterns" #287

query25 = "What are the main reasons why FIS and Fiserv are seeing significant stock growth, and what outlook do analysts have for these companies in the coming years?" #311

query = query11

In [122]:
df = preprocess_df(df)

AttributeError: 'list' object has no attribute 'translate'

In [126]:
bm25_instance = bm25.BM25(df["content"],k1=1.5, b=0.75)
print(queries)
top1 = 0
top3 = 0
top5 = 0
for idx, row in queries.iterrows():
    query = row["query"]
    doc_index = row["doc_index"]
    query = preprocess_query(query)
    ranked_indices, scores = rank_query_bm25(bm25_instance, query, doc_index,  summarize_docs=False, only_scores=True)
    if doc_index in ranked_indices:
        top5 += 1
        if doc_index in ranked_indices[:3]:
            top3 += 1
            if doc_index in ranked_indices[:1]:
                top1+=1
                print(scores[:2])
            else:
                rank_query_bm25(bm25_instance, query, doc_index,  summarize_docs=False, only_scores=False)
        else:
            rank_query_bm25(bm25_instance, query, doc_index,  summarize_docs=False, only_scores=False)
    else:
        rank_query_bm25(bm25_instance, query, doc_index,  summarize_docs=False, only_scores=False)
print(top5)
print(top3)
print(top1)

                                                query  doc_index
0   Why is Australia's monetary policy still seen ...        551
1   What's Palo Alto Networks' revenue forecast fo...        400
2   How could institutional investments affect Bit...        377
3   Hey, I've been hearing that some AI companies ...        240
4   What impact could increased tariffs have on th...        516
5   Why is the U.S. government reducing Intel's ch...        345
6   What is Israel considering regarding a cease-f...        213
7   How have Baidu's shares performed this year, a...        370
8   What challenges does the U.S. face in reaching...        554
9   Why are creditors frustrated with the stalled ...        276
10  How did Zoom's third-quarter results compare t...         51
11  How do geopolitical risks, valuation mismatche...        299
12  Why is Ford planning to reduce its workforce i...        567
13  What's going on with Ricardo Salinas Pliego's ...        234
14  Why do experts believ

In [11]:
count=0
for i in range(len(df)):
    article = df.iloc[i]["content"]
    if len(article) < 100:
        count+=1
        print(article)
print(count)

Equity Markets Fall Intraday Ahead of Nvidia Earnings Sign in to access your portfolio
1


In [111]:
df.iloc[55]["content"]

['bloomberg',
 'softbank',
 'group',
 'corp',
 'aims',
 'increase',
 'stake',
 'openai',
 'acquiring',
 'billion',
 'shares',
 'startup’s',
 'employees',
 'according',
 'people',
 'familiar',
 'matter',
 'read',
 'bloomberg',
 'new',
 'york',
 'city’s',
 '‘living',
 'breakwaters’',
 'brace',
 'stormier',
 'seas',
 'italy’s',
 'motor',
 'city',
 'car',
 'free',
 'options',
 'growing',
 'kansas',
 'city',
 'first',
 'ever',
 'stadium',
 'designed',
 'women’s',
 'sports',
 'takes',
 'field',
 'nyc',
 'underground',
 'steam',
 'system',
 'may',
 'key',
 'greener',
 'future',
 'nyc',
 'gets',
 'historic',
 'push',
 '80',
 '000',
 'homes',
 'billion',
 'pledge',
 'japanese',
 'company',
 'make',
 'tender',
 'offer',
 'stock',
 'allowing',
 'openai',
 'employees',
 'cash',
 'shares',
 'choose',
 'softbank',
 'contributed',
 '500',
 'million',
 'openai’s',
 'billion',
 'fundraising',
 'round',
 'october',
 'pushed',
 'larger',
 'allocation',
 'time',
 'said',
 'one',
 'people',
 'asking',
 'na